In [ ]:
from DeepCpf1_processing import *
import os
from os.path import exists

#Parameters
nt_order = "ATGC"
DATA_len = 34
DATA_offset = 0

#TensorFlow config
conf = tf.ConfigProto()
conf.gpu_options.per_process_gpu_memory_fraction = 0.05
conf.gpu_options.allow_growth = True
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
# -----------------------------------------------------
# Seq-deepCpf1 PreTrain
# -----------------------------------------------------
if exists("model_checkpoints/PreTrain-Final.meta"):
    sys.exit()

FILE = open("PreTrain_DATA.txt", "r")
data = FILE.readlines()
DATA_X, DATA_Y, _ = PREPROCESS(data, DATA_len, DATA_offset)
FILE.close()

train_n = int(DATA_X.shape[0] * 0.9)
TRAIN_DATA_X, TRAIN_DATA_Y = DATA_X[:train_n], DATA_Y[:train_n]
VAL_DATA_X, VAL_DATA_Y = DATA_X[train_n:], DATA_Y[train_n:]

tf.reset_default_graph()
with tf.Session(config=conf) as sess:
    model = Seq_deepCpf1()
    sess.run(tf.global_variables_initializer())
    Model_PreTrain(sess, model, TRAIN_DATA_X, TRAIN_DATA_Y, VAL_DATA_X, VAL_DATA_Y)

In [ ]:
# -----------------------------------------------------
# DepCpf1 FineTune
# -----------------------------------------------------
if exists("model_checkpoints/FineTune-Final.meta"):
    sys.exit()

FILE = open("FineTune_DATA.txt", "r")
data = FILE.readlines()
FILE.close()

TRAIN_DATA_X, TRAIN_DATA_Y, TRAIN_DATA_CA = PREPROCESS(data, DATA_len, DATA_offset)

tf.reset_default_graph()  
with tf.Session(config=conf) as sess:    
    model = DeepCpf1()
    sess.run(tf.global_variables_initializer())
    
    Var_list = {v.op.name: v for v in tf.trainable_variables() if "CA" not in v.op.name}
    saver = tf.train.Saver(Var_list)
    saver.restore(sess, "model_checkpoints/PreTrain-Final")
    
    Model_FineTune(sess, model, TRAIN_DATA_X, TRAIN_DATA_Y, TRAIN_DATA_CA)    

In [ ]:
# -----------------------------------------------------
# DeepCpf1 / Seq-deepCpf1 Prediction
# -----------------------------------------------------
TEST_LIST = ["TEST_DATA"]

for TEST in TEST_LIST:
    FILE = open("%s.txt" % TEST, "r")
    data = FILE.readlines()
    FILE.close()

    TEST_DATA_X, _, TEST_DATA_CA = PREPROCESS(data, DATA_len, DATA_offset)
    TEST_PREDICTIONS = []

    for mode in ["DeepCpf1", "Seq-deepCpf1"]:
        tf.reset_default_graph()  
        with tf.Session(config=conf) as sess:
            model = Model_Load(sess, mode)
            TEST_PREDICTIONS.append(Model_Prediction(sess, model, mode, TEST_DATA_X, TEST_DATA_CA))

    OUT = open("%s_Predictions.txt" % TEST, "w")
    OUT.write("%s\tDeepCpf1\tSeq-deepCpf1\n" % data[0].strip())
    for l in range(1, len(data)):
        OUT.write("%s\t%f\t%f\n" % (data[l].strip(), TEST_PREDICTIONS[0][l-1], TEST_PREDICTIONS[1][l-1]))
    OUT.close()